In [1]:
import pandas as pd
from pandas import json_normalize
import json
import re
from ast import literal_eval

In [2]:
df = pd.read_json('/content/drive/MyDrive/sampleNestedFile.json')
display(df)

,id,type,name,ppu,batters,topping
0,1,donut,Cake,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."
1,2,donut,Raised,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'}]}","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."
2,3,donut,Old Fashioned,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[{'id': '5001', 'type': 'None'}, {'id': '5002'..."


In [3]:
df_topping = json_normalize(df['topping'])
display(df_topping)

,0,1,2,3,4,5,6
0,"{'id': '5001', 'type': 'None'}","{'id': '5002', 'type': 'Glazed'}","{'id': '5005', 'type': 'Sugar'}","{'id': '5007', 'type': 'Powdered Sugar'}","{'id': '5006', 'type': 'Chocolate with Sprinkl...","{'id': '5003', 'type': 'Chocolate'}","{'id': '5004', 'type': 'Maple'}"
1,"{'id': '5001', 'type': 'None'}","{'id': '5002', 'type': 'Glazed'}","{'id': '5005', 'type': 'Sugar'}","{'id': '5003', 'type': 'Chocolate'}","{'id': '5004', 'type': 'Maple'}",None,None
2,"{'id': '5001', 'type': 'None'}","{'id': '5002', 'type': 'Glazed'}","{'id': '5003', 'type': 'Chocolate'}","{'id': '5004', 'type': 'Maple'}",None,None,None


In [5]:
# Define a regular expression pattern to match integers after "id"
pattern = re.compile(r'"id":\s*"(\d+)"')
df_topping_cols = pd.DataFrame()
df_topping_cols['col1'] = ''
# Extract integers after "id" using the pattern
for i in range(len(df)):
  result = [int(pattern.search(str(item).replace("'", '"')).group(1)) if pattern.search(str(item).replace("'", '"')) else None for item in df['topping'][i]]

  df['topping'][i] = result

<ipython-input-5-4711c6bd7c8c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topping'][i] = result


In [6]:
display(df)

,id,type,name,ppu,batters,topping
0,1,donut,Cake,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[5001, 5002, 5005, 5007, 5006, 5003, 5004]"
1,2,donut,Raised,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'}]}","[5001, 5002, 5005, 5003, 5004]"
2,3,donut,Old Fashioned,0.55,"{'batter': [{'id': '1001', 'type': 'Regular'},...","[5001, 5002, 5003, 5004]"


In [7]:
df_single_column = df_topping.stack().reset_index(drop=True).to_frame()
df_single_column = df_single_column[0].apply(str).unique()

In [8]:
display(df_single_column)

array(["{'id': '5001', 'type': 'None'}",
       "{'id': '5002', 'type': 'Glazed'}",
       "{'id': '5005', 'type': 'Sugar'}",
       "{'id': '5007', 'type': 'Powdered Sugar'}",
       "{'id': '5006', 'type': 'Chocolate with Sprinkles'}",
       "{'id': '5003', 'type': 'Chocolate'}",
       "{'id': '5004', 'type': 'Maple'}"], dtype=object)

In [9]:
data_list = [literal_eval(item) for item in df_single_column]

# Create a DataFrame
df_topping_list = pd.DataFrame(data_list)

In [10]:
display(df_topping_list)

,id,type
0,5001,None
1,5002,Glazed
2,5005,Sugar
3,5007,Powdered Sugar
4,5006,Chocolate with Sprinkles
5,5003,Chocolate
6,5004,Maple


In [11]:
df_batter = pd.DataFrame(df['batters'])
display(df_batter)

,batters
0,"{'batter': [{'id': '1001', 'type': 'Regular'},..."
1,"{'batter': [{'id': '1001', 'type': 'Regular'}]}"
2,"{'batter': [{'id': '1001', 'type': 'Regular'},..."


In [13]:
for i in range(len(df)):
  x = df['batters'][i].values()
  x = list(x)[0]
  df_batter['batters'][i] = x

df_batter_normalized = json_normalize(df_batter['batters'])
display(df_batter_normalized)

,0,1,2,3
0,"{'id': '1001', 'type': 'Regular'}","{'id': '1002', 'type': 'Chocolate'}","{'id': '1003', 'type': 'Blueberry'}","{'id': '1004', 'type': 'Devil's Food'}"
1,"{'id': '1001', 'type': 'Regular'}",None,None,None
2,"{'id': '1001', 'type': 'Regular'}","{'id': '1002', 'type': 'Chocolate'}",None,None


In [14]:
# Define a regular expression pattern to match integers after "id"
pattern = re.compile(r'"id":\s*"(\d+)"')

res =[]
df_batter_cols = pd.DataFrame()
df_batter_cols['col1'] = ''
# Extract integers after "id" using the pattern
for i in range(len(df)):
  result = [int(pattern.search(str(item).replace("'", '"')).group(1)) if pattern.search(str(item).replace("'", '"')) else None for item in df_batter['batters'][i]]

  df['batters'][i] = result

<ipython-input-14-cbb4c3b6baaa>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['batters'][i] = result


In [15]:
df_batter_column = df_batter_normalized.stack().reset_index(drop=True).to_frame()
df_batter_column = df_batter_column[0].apply(str).unique()
display(df_batter_column)

array(["{'id': '1001', 'type': 'Regular'}",
       "{'id': '1002', 'type': 'Chocolate'}",
       "{'id': '1003', 'type': 'Blueberry'}",
       '{\'id\': \'1004\', \'type\': "Devil\'s Food"}'], dtype=object)

In [16]:
data_list_batter = [literal_eval(item) for item in df_batter_column]

# Create a DataFrame
df_batter_list = pd.DataFrame(data_list_batter)

In [17]:
display(df_batter_list)

,id,type
0,1001,Regular
1,1002,Chocolate
2,1003,Blueberry
3,1004,Devil's Food


In [18]:
display(df)

,id,type,name,ppu,batters,topping
0,1,donut,Cake,0.55,"[1001, 1002, 1003, 1004]","[5001, 5002, 5005, 5007, 5006, 5003, 5004]"
1,2,donut,Raised,0.55,[1001],"[5001, 5002, 5005, 5003, 5004]"
2,3,donut,Old Fashioned,0.55,"[1001, 1002]","[5001, 5002, 5003, 5004]"


In [66]:
# Get the toppings for donut having id = 2
df_id = df.loc[(df['id']==2)]
df_id = df_id['topping']


y = pd.DataFrame()
y['type'] = ''
for i in df_id:
  for j in range(len(i)):
    x = df_topping_list['type'].loc[(df_topping_list['id']==str(i[j]))]

    if not x.empty:
            y = pd.concat([y, pd.DataFrame({'type': x.values})], ignore_index=True)
display(y)



,type
0,None
1,Glazed
2,Sugar
3,Chocolate
4,Maple
